# Plotting graphs in Datashader with edge bundling

[Datashader](http://datashader.readthedocs.org) provides general-purpose support for plotting very large collections of points, line segments, and gridded data, using a convenient API provided by [HoloViews](http://holoviews.org) and  an interactive user interface provided by [Bokeh](http://bokeh.pydata.org). While these facilities can be used to plot any sort of data, graph data (networks of nodes connected by edges) presents some special challenges.  At a minimum, the nodes and edges must be mapped into specific locations and shapes in a 2D plane before they can be visualized, which is an [open-ended problem involving a complex set of tradeoffs and complications](http://www.hiveplot.com).

Here, we will assume that the nodes have already been laid out in 2D, whether because they have such structure inherently (such as connections between geographic locations), or an arbitrary random or geometric layout has been selected manually, or the layout has been determined by an external tool (such as a force-directed algorithm laying out similarly connected nodes in similar locations).

First, we'll import the packages we are using and set up some defaults.

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
import fastparquet as fp
import datashader as ds

from colorcet import fire
from datashader.bundling import directly_connect_edges, hammer_bundle
from datashader.utils import export_image

from holoviews.operation.datashader import aggregate, shade, datashade, dynspread
from holoviews.operation import decimate

from dask.distributed import Client
client = Client()

hv.notebook_extension('bokeh','matplotlib')

decimate.max_samples=20000
dynspread.threshold=0.01
datashade.cmap=fire[40:]
sz = dict(width=150,height=150)

%opts RGB [xaxis=None yaxis=None show_grid=False bgcolor="black"]

# Random graph

As a first example, let's generate a random graph, with 5000 points normally distributed around the origin and 10000 random connections between them:

In [ ]:
np.random.seed(0)

nodes = hv.Points(pd.DataFrame(np.random.randn(5000, 2), columns=['x', 'y']))
nodes.data.tail()

In [ ]:
edges = hv.Curve(pd.DataFrame(np.random.randint(len(nodes.data), size=(10000, 2)),
                              columns=['source', 'target']))
edges.data.tail()

Here you can see that the nodes list is a columnar dataframe with an index value and an (x,y) location for every node.  The edges list is a columnar dataframe listing the index of the source and target in the nodes dataframe.  

We can now visualize this data using some graph-specific helper functions in Datashader, with results packaged as HoloViews objects that display using Bokeh:

In [ ]:
%%opts RGB [width=400 height=400]

%time direct  = hv.Curve(directly_connect_edges(nodes.data, edges.data))
%time bundled = hv.Curve(hammer_bundle(nodes.data, edges.data))

(datashade(direct)  * dynspread(datashade(nodes,cmap=["cyan"]),threshold=0.01)).relabel("Directly connected") + \
(datashade(bundled) * dynspread(datashade(nodes,cmap=["cyan"]),threshold=0.01)).relabel("Bundled")

In these plots, you can select the Bokeh "wheel zoom" from the tool palette and zoom or pan on the plot.  On a static site like anaconda.org the image will be rescaled blockily as you zoom in, not revealing any new data, but with a live server it will be dynamically re-rendered to show more detailed structure each time. Here the nodes are shown as cyan-colored dots, and the connections are lines that show up as dark red for a single line and gradually brighter colors as more lines are overlaid.  

The "Directly Connected" plot shows a single line segment per edge, which is very quick (scaling in computational complexity with the number of edges) but has so much overlap that it is difficult to see the underlying structure.

Conversely, if we render the edges as curves by using an edge bundling algorithm (a variant of [Hurter, Ersoy, & Telea, ECV-2012](http://www.cs.rug.nl/~alext/PAPERS/EuroVis12/kdeeb.pdf)), we can see structure at a local scale.  (Of course, the large-scale structure in this case is random, but this technique magnifies small fluctuations, which in real datasets presumably reflect actual structure of the data.)

The specific patterns that develop in the bundling process depend on the bundling parameters, particularly the initial bandwidth over which grouping is done, and the rate of decay of this bandwidth:

In [ ]:
nodes_ds = dynspread(datashade(nodes,cmap=["cyan"]),threshold=0.01)

%time bundled         = hv.Curve(hammer_bundle(nodes.data, edges.data))
%time bundled_decay04 = hv.Curve(hammer_bundle(nodes.data, edges.data, decay=0.4))
%time bundled_decay10 = hv.Curve(hammer_bundle(nodes.data, edges.data, decay=1.0))
%time bundled_bw01    = hv.Curve(hammer_bundle(nodes.data, edges.data, initial_bandwidth=0.01))
%time bundled_bw30    = hv.Curve(hammer_bundle(nodes.data, edges.data, initial_bandwidth=0.30))

In [ ]:
%%opts RGB [width=250 height=250] {+framewise}

((datashade(bundled_bw01,**sz)    * nodes_ds).relabel("Bundled with bw=0.01") + \
 (datashade(bundled,**sz)         * nodes_ds).relabel("Bundled with bw=0.05") + \
 (datashade(bundled_bw30,**sz)    * nodes_ds).relabel("Bundled with bw=0.30") + \
 (datashade(bundled_decay04,**sz) * nodes_ds).relabel("Bundled with decay=0.1") + \
 (datashade(bundled,**sz)         * nodes_ds).relabel("Bundled with decay=0.7") + \
 (datashade(bundled_decay10,**sz) * nodes_ds).relabel("Bundled with decay=1.0")).cols(3)

# Other standard graph types using NetworkX

The separate [NetworkX](https://networkx.readthedocs.io) package can be used to create a variety of standard [graph structures](https://networkx.readthedocs.io/en/stable/reference/generators.html), so that you can see how bundling works across different graph types.  Here, we lay each of them out in a circular pattern using NetworkX, but note as always that results will differ dramatically depending on the layout, so it is important to choose a good layout first.

In [ ]:
import networkx as nx

def nx_plot(graph):
    print(graph.name, len(graph.edges()))
    layout = nx.circular_layout(graph)
    data = [[node, *layout[node]] for node in graph.nodes_iter()]

    nodes_df = pd.DataFrame(data, columns=['id', 'x', 'y'])
    nodes_df.set_index('id', inplace=True)
    nodes = hv.Points(nodes_df)

    edges = hv.Points(pd.DataFrame(graph.edges_iter(), columns=['source', 'target']))
    direct = hv.Curve(directly_connect_edges(nodes.data, edges.data))

    bundled_bw005 = hv.Curve(hammer_bundle(nodes.data, edges.data))
    bundled_bw030 = hv.Curve(hammer_bundle(nodes.data, edges.data, initial_bandwidth=0.30))

    return (datashade(direct, **sz)        * nodes).relabel(graph.name) + \
           (datashade(bundled_bw005, **sz) * nodes).relabel("Bundled bw=0.05") + \
           (datashade(bundled_bw030, **sz) * nodes).relabel("Bundled bw=0.30")

In [ ]:
%%time
%%opts RGB [width=200 height=200] {+framewise}
n=100

hv.Layout([nx_plot(g) for g in
           [nx.complete_graph(n), nx.lollipop_graph(n, 5),     nx.barbell_graph(n,2),
            nx.ladder_graph(n),   nx.circular_ladder_graph(n), nx.star_graph(n),
            nx.cycle_graph(n)]]).cols(3)

As you can see, both bundled and unbundled representations reflect important aspects of the graph structure, but the bundling results do depend on the parameters chosen.  Bundling is also very computationally expensive, with unbundled plots much less expensive to prepare.

# UK research collaboration example

The above examples use artificial datasets, but each real-world dataset will have its own specific properties. Here, let's look at one example dataset of 600,000 collaborations between 15000 UK research institutions, previously laid out using a force-directed algorithm by [Ian Calvert](https://www.digital-science.com/people/ian-calvert):

In [ ]:
r_nodes_file = 'data/calvert_uk_research2017_nodes.snappy.parq'
r_edges_file = 'data/calvert_uk_research2017_edges.snappy.parq'

r_nodes = hv.Points(fp.ParquetFile(r_nodes_file).to_pandas(index='id'), label="Nodes")
r_edges = hv.Curve( fp.ParquetFile(r_edges_file).to_pandas(index='id'), label="Edges")
len(r_nodes),len(r_edges)

We can render each collaboration as a single-line direct connection, but the result is a dense tangle:

In [ ]:
%%opts RGB [tools=["hover"] width=400 height=400] 

%time r_direct = hv.Curve(directly_connect_edges(r_nodes.data, r_edges.data),label="Direct")

dynspread(datashade(r_nodes,cmap=["cyan"])) + \
datashade(r_direct)

Detailed substructure of this graph becomes visible after bundling, which takes several minutes even using multiple cores with Dask:

In [ ]:
%time r_bundled = hv.Curve(hammer_bundle(r_nodes.data, r_edges.data),label="Bundled")

In [ ]:
%%opts RGB [tools=["hover"] width=400 height=400] 

dynspread(datashade(r_nodes,cmap=["cyan"])) + datashade(r_bundled)

Zooming into these plots reveals interesting patterns, but immediately one then wants to ask what the various groupings of nodes might represent. With a small number of nodes or a small number of categories one could color-code the dots (using datashader's categorical color coding support), but here we just have thousands of indistinguishable dots. Instead, let's use hover information so the viewer can see the identity of each node.  

To give something useful to hover, let's load the names of each institution in the researcher list and merge that with our existing layout data:

In [ ]:
node_names = pd.read_csv("data/calvert_uk_research2017_nodes.csv", index_col="node_id", usecols=["node_id","name"])
node_names = node_names.rename(columns={"name": "Institution"})
node_names

r_nodes_named = pd.merge(r_nodes.data, node_names, left_index=True, right_index=True)
r_nodes_named.tail()

We can now overlay a set of points on top of the datashaded edges, which will provide hover information for each node.  Here, the entire set of 15000 nodes would be reasonably feasible to plot, but to show how to work with larger datasets we wrap the `hv.Points()` call with `decimate` so that only a finite subset of the points will be shown at any one time. If a node of interest is not visible in a particular zoom, then you can simply zoom in on that region; at some point the number of visible points will be below the specified decimate limit and the required point should be revealed.

In [ ]:
%%opts Points (color="cyan") [tools=["hover"] width=900 height=650] 
datashade(r_bundled, width=900, height=650) * decimate(hv.Points(r_nodes_named),max_samples=2000)

If you click around and hover, you should see interesting groups of nodes, and can then set up further interactive tools using [HoloViews' stream support](http://holoviews.org/Tutorials/Streams.html) to reveal aspects relevant to your research interests or questions.

As you can see, datashader lets you work with very large graph datasets, though there are a number of decisions to make by trial and error, you do have to be careful when doing computationally expensive operations like edge bundling, and interactive information will only be available for a limited subset of the data at any one time.